In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd


In [2]:
from sklearn.preprocessing import MinMaxScaler


In [3]:
from sklearn.preprocessing import OneHotEncoder


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
# Check available CUDA devices and memory
if torch.cuda.is_available():
    num_devices = torch.cuda.device_count()
    for i in range(num_devices):
        device = torch.cuda.device(i)
        total_mem = torch.cuda.get_device_properties(i).total_memory / 1024**3  # Convert to GB
        allocated_mem = torch.cuda.memory_allocated(i) / 1024**3  # Convert to GB
        free_mem = total_mem - allocated_mem
        
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"Total Memory: {total_mem:.1f}GB")
        print(f"Allocated Memory: {allocated_mem:.1f}GB")
        print(f"Free Memory: {free_mem:.1f}GB")
        
        if free_mem < 8:
            print(f"Warning: GPU {i} has less than 8GB of free VRAM!")
        else:
            print(f"Using GPU {i} with {free_mem:.1f}GB free VRAM")
            break 
    device = torch.device(f"cuda:{i}")
else:
    print("Warning: No CUDA devices available - running on CPU only")
    device = torch.device("cpu")


GPU 0: NVIDIA GeForce RTX 4070 Ti
Total Memory: 12.0GB
Allocated Memory: 0.0GB
Free Memory: 12.0GB
Using GPU 0 with 12.0GB free VRAM


In [ ]:
# loss function and optimizer
loss_fn = nn.MSELoss()  # mean square error
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [6]:
import torch
from torch.utils.data import Dataset

class SolarDataset(Dataset):
    def __init__(self, net_load, solar_gen=None, sequence_length=24):
        """
        Args:
            net_load (numpy array or list): Net load values over time.
            weather (numpy array or list): Corresponding weather features.
            solar_gen (numpy array or list, optional): Solar generation values for supervised learning.
            sequence_length (int): Number of past time steps to include in each input sequence.
        """
        self.net_load = torch.tensor(net_load, dtype=torch.float32).unsqueeze(-1)
        #self.weather = torch.tensor(weather, dtype=torch.float32)
        self.solar_gen = torch.tensor(solar_gen, dtype=torch.float32) if solar_gen is not None else None
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.net_load) - self.sequence_length  # Total number of samples

    def __getitem__(self, idx):
        net_seq = self.net_load[idx:idx+self.sequence_length]
        #weather_seq = self.weather[idx:idx+self.sequence_length]

        if self.solar_gen is not None:
            target = self.solar_gen[idx+self.sequence_length]  # Next time step's solar generation
            return net_seq, target
        else:
            return net_seq  # Used for inference

In [7]:
df_train = pd.read_csv("./model_input_data/2010-2012.csv")
df_test = pd.read_csv("./model_input_data/2012-2013.csv")

In [8]:
print(df_train.head())

   Customer       date  Time     GC             datetime   GG   CL     NL  \
0         6  01-Jul-10  0:00  0.036  2010-07-01 00:00:00  0.0  0.0  0.036   
1         6  01-Jul-10  0:30  0.044  2010-07-01 00:30:00  0.0  0.0  0.044   
2         6  01-Jul-10  1:00  0.041  2010-07-01 01:00:00  0.0  0.0  0.041   
3         6  01-Jul-10  1:30  0.047  2010-07-01 01:30:00  0.0  0.0  0.047   
4         6  01-Jul-10  2:00  0.046  2010-07-01 02:00:00  0.0  0.0  0.046   

   Row Quality_x  Row Quality_y  Row Quality  
0            NaN            NaN          NaN  
1            NaN            NaN          NaN  
2            NaN            NaN          NaN  
3            NaN            NaN          NaN  
4            NaN            NaN          NaN  


In [9]:
print(df_test.head())

   Customer       date  Row Quality_x  Time     GC             datetime  \
0       233  1/07/2012            NaN  0:00  0.040  2012-07-01 00:00:00   
1       233  1/07/2012            NaN  0:30  0.040  2012-07-01 00:30:00   
2       233  1/07/2012            NaN  1:00  0.034  2012-07-01 01:00:00   
3       233  1/07/2012            NaN  1:30  0.039  2012-07-01 01:30:00   
4       233  1/07/2012            NaN  2:00  0.039  2012-07-01 02:00:00   

   Row Quality_y     GG     NL  Row Quality  CL  
0            NaN  0.006  0.034          NaN NaN  
1            NaN  0.000  0.040          NaN NaN  
2            NaN  0.006  0.028          NaN NaN  
3            NaN  0.000  0.039          NaN NaN  
4            NaN  0.000  0.039          NaN NaN  


In [10]:
# Selecting relevant columns
net_load_train = df_train["NL"].values
#weather_train = train_df[["Temperature", "Solar Irradiance", "Humidity"]].values  # Adjust based on dataset
solar_gen_train = df_train["GG"].values if "GG" in df_train else None

net_load_test = df_test["NL"].values
#weather_test = test_df[["Temperature", "Solar Irradiance", "Humidity"]].values
solar_gen_test = df_test["GG"].values if "GG" in df_test else None

In [11]:
# Convert to PyTorch Dataset
#train_dataset = SolarDataset(net_load_train, weather_train, solar_gen_train, sequence_length=24)
#test_dataset = SolarDataset(net_load_test, weather_test, solar_gen_test, sequence_length=24)
# Convert to PyTorch Dataset
training_data = SolarDataset(net_load_train, solar_gen_train, sequence_length=24)
test_data = SolarDataset(net_load_test, solar_gen_test, sequence_length=24)

In [12]:
# Check dataset length
print(f"Total samples available: {len(training_data)}")


Total samples available: 10518216


In [13]:
net_seq, target = training_data[0]  # Get the first sample

print("Net Load Sequence Shape:", net_seq.shape)
#print("Weather Sequence Shape:", weather_seq.shape)
print("Target (Solar Generation) Value:", target)

Net Load Sequence Shape: torch.Size([24, 1])
Target (Solar Generation) Value: tensor(0.6250)


In [14]:

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
#test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True/False)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=False)

In [15]:
for net_seq, target in train_dataloader:
    print("Batch Net Load Shape:", net_seq.shape)
    #print("Batch Weather Shape:", weather_seq.shape)
    print("Batch Target Shape:", target.shape)
    break  # Only print first batch


Batch Net Load Shape: torch.Size([64, 24, 1])
Batch Target Shape: torch.Size([64])


In [16]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

In [17]:

# Initialize model
#input_size = weather_train.shape[1]  # Number of weather features
input_size = 1  # Assuming net load is a single feature
model = LSTM(input_size, hidden_size=64, num_layers=2, output_size=1).to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
for epoch in range(10):  # Set desired number of epochs
    model.train()
    epoch_loss = 0

    for net_seq, target in train_dataloader:
        net_seq, target = net_seq.to(device), target.to(device)
        optimizer.zero_grad()

        # Forward pass
        #output = model(weather_seq)  # Model learns from weather data
        output = model(net_seq)

        # Compute loss
        loss = criterion(output.squeeze(), target)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {epoch_loss / len(train_dataloader)}")

Epoch 1, Loss: 0.009413103695231744
Epoch 2, Loss: 0.008707379919137758
Epoch 3, Loss: 0.008475266136201073
Epoch 4, Loss: 0.008334303024419659
Epoch 5, Loss: 0.00823465331809194
Epoch 6, Loss: 0.008145611358767296
Epoch 7, Loss: 0.008078296554285788
Epoch 8, Loss: 0.008024703321467527
Epoch 9, Loss: 0.007965441006583916
Epoch 10, Loss: 0.00791951701160443


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, target_column):
        self.data = pd.read_csv(csv_file)
        self.target_column = target_column

        # Normalize numerical features
        self.scaler = MinMaxScaler()
        self.data[self.data.select_dtypes(include=['float64', 'int64']).columns] = self.scaler.fit_transform(
            self.data.select_dtypes(include=['float64', 'int64']).values)

        # One-hot encode categorical features
        self.encoder = OneHotEncoder(sparse=False)
        categorical_cols = self.data.select_dtypes(include=['object']).columns
        encoded_categorical_data = self.encoder.fit_transform(self.data[categorical_cols])
        encoded_categorical_df = pd.DataFrame(encoded_categorical_data, columns=self.encoder.get_feature_names_out(categorical_cols))
        
        # Combine normalized numerical data with one-hot encoded categorical data
        self.data = pd.concat([self.data.drop(columns=categorical_cols), encoded_categorical_df], axis=1)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        target = row[self.target_column]
        features = row.drop(self.target_column).values.astype(float)
        return torch.tensor(features, dtype=torch.float32), torch.tensor(target, dtype=torch.float32)

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
import torch.nn as nn
import torch.optim as optim
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Model, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.lstm = nn.LSTMCell(self.input_size, self.hidden_size)
        self.linear = nn.Linear(self.hidden_size, self.output_size)
    def forward(self, input, future=0, y=None):
        outputs = []
        # reset the state of LSTM
        # the state is kept till the end of the sequence
        h_t = torch.zeros(input.size(0), self.hidden_size, dtype=torch.float32)
        c_t = torch.zeros(input.size(0), self.hidden_size, dtype=torch.float32)
        for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
            h_t, c_t = self.lstm(input_t, (h_t, c_t))
            output = self.linear(h_t)
            outputs += [output]
        for i in range(future):
            if y is not None and random.random() > 0.5:
                output = y[:, [i]]  # teacher forcing
            h_t, c_t = self.lstm(output, (h_t, c_t))
            output = self.linear(h_t)
            outputs += [output]
        outputs = torch.stack(outputs, 1).squeeze(2)
        return outputs